# Data Preprocessing - Standardization
This tutorial explains how to preprocess data using the Pandas library. Preprocessing is the process of doing a pre-analysis of data, in order to transform them into a standard and normalised format. Preprocessing involves the following aspects:
* missing values
* data formatting
* data normalisation
* data standardisation
* data binning
In this tutorial we deal only with standardization. Standardization is often confused with normalization, however
they refer to different things. Normalization involves adjusting values measured on different scales to a common scale, while standardization transforms data to have a mean of zero and a standard deviation of 1. 
Standardization is also done through a z-score transformation, where the new value is calculated as the difference between the current value and the average value, divided by the standard deviation. 

Z-score is a statistical measure that specifies how far is a single data point from the rest of the dataset. As highlighted by Mahbubul Alam in [his article](https://towardsdatascience.com/z-score-for-anomaly-detection-d98b0006f510), z-score can be used to detect outliers in a dataset.

Z-score can be calculated manually as described in [my previous post](https://towardsdatascience.com/data-preprocessing-with-python-pandas-part-3-normalisation-5b5392d27673). However, in this tutorial I will show you how to calculate z-score using some functions from the `scipy.stats` library.

In this tutorial we consider two types of standardizations:
* z-score
* z-map

## Data Import
As example dataset, in this tutorial we consider the dataset provided by the Italian Protezione Civile, related to the number of COVID-19 cases registered since the beginning of the COVID-19 pandemic. The dataset is updated daily and can be downloaded from [this link](https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv).

First of all, we need to import the Python `pandas` library and read the dataset through the `read_csv()` function. Then we can drop all the columns with `NaN` values. This is done through `dropna()` function. 

In [79]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv')
df.dropna(axis=1,inplace=True)
df.tail(10)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,totale_casi,tamponi
6206,2020-12-15T17:00:00,ITA,21,P.A. Bolzano,46.499335,11.356624,208,24,232,10398,10630,18,88,15653,657,26940,337081
6207,2020-12-15T17:00:00,ITA,22,P.A. Trento,46.068935,11.121231,412,53,465,2242,2707,153,377,15344,795,18846,414855
6208,2020-12-15T17:00:00,ITA,1,Piemonte,45.073274,7.680687,3761,266,4027,46943,50970,-2366,1106,128989,7213,187172,1784446
6209,2020-12-15T17:00:00,ITA,16,Puglia,41.125596,16.867367,1530,187,1717,50558,52275,243,1023,20364,2012,74651,910661
6210,2020-12-15T17:00:00,ITA,20,Sardegna,39.215312,9.110616,570,58,628,14756,15384,23,231,10753,600,26737,426556
6211,2020-12-15T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1225,185,1410,34559,35969,128,1087,42192,2030,80191,1098601
6212,2020-12-15T17:00:00,ITA,9,Toscana,43.769231,11.255889,1156,214,1370,14894,16264,-1215,332,93619,3238,113121,1733863
6213,2020-12-15T17:00:00,ITA,10,Umbria,43.106758,12.388247,288,46,334,4226,4560,-175,179,21323,535,26418,460748
6214,2020-12-15T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,74,6,80,447,527,-61,20,6022,356,6905,58718
6215,2020-12-15T17:00:00,ITA,5,Veneto,45.434905,12.338452,2694,346,3040,89650,92690,162,3320,99108,4992,196790,3023689


## z-score
The new value is calculated as the difference between the current value and the average value, divided by the standard deviation. For example, we can calculate the z-score of the column `deceduti`. We can use the `zscore()` function of the `scipy.stats` library.

In [82]:
from scipy.stats import zscore
df['zscore-deceduti'] = zscore(df['deceduti'])

## z-map
The new value is calculated as the difference between the current value and the average value of a comparison array, divided by the standard deviation of a comparison array. For example, we can calculate the z-map of the column `deceduti`, using the column `terapia_intensiva` as comparison array. We can use the `zmap()` function of the `scipy.stats` library.

In [83]:
from scipy.stats import zmap
zmap(df['deceduti'], df['terapia_intensiva'])

array([-0.42939174, -0.42939174, -0.42939174, ...,  3.47300249,
        2.16734162, 35.98322884])

## Detect outliers
Standardization can be used to detect and delete outliers. For example, a threshold can be defined to specify which values can be considered as outliers. In this example, we set `threshold = 2`. We can add a new column to the dataframe, called `outliers` which is set to `True` if the value is less than `-2` or greater than `2`. We use the `numpy` function `where()` to perform comparisons.

In [84]:
threshold = 2
df['outliers'] = np.where((df['zscore-deceduti'] - threshold > 0), True, np.where(df['zscore-deceduti'] + threshold < 0, True, False)) 

Now, we can remove outliers, using the `drop()` function.

In [93]:
df.drop(df[df['outliers'] == True].index,inplace=True)

In [94]:
df.shape

(5960, 19)